In [1]:
testdata = {'A': 729, 'B': 0, 'C': 0, 'Program': (0,1,5,4,3,0)}

In [2]:
data = {'A': 33940147, 'B': 0, 'C': 0, 'Program': (2,4,1,5,7,5,1,6,4,2,5,5,0,3,3,0)}

## Part 1 ##

In [3]:
def regdv(outregister, operand, registers, insptr, outs):
    numerator = registers['A']
    match operand:
        case 0 | 1 | 2 | 3:
            power = operand
        case 4:
            power = registers['A']
        case 5:
            power = registers['B']
        case 6:
            power = registers['C']
        case _:
            raise ValueError(f'Invalid operand: {operand}')
    val = numerator//(2**power)
    registers[outregister] = val
    return insptr+2

In [4]:
def adv(operand, registers, insptr, outs):
    return regdv('A', operand, registers, insptr, outs)
def bdv(operand, registers, insptr, outs):
    return regdv('B', operand, registers, insptr, outs)
def cdv(operand, registers, insptr, outs):
    return regdv('C', operand, registers, insptr, outs)

In [5]:
def bxl(operand, registers, insptr, outs):
    registers['B'] = registers['B'] ^ operand
    return insptr+2

In [6]:
def bst(operand, registers, insptr, outs):
    match operand:
        case 0 | 1 | 2 | 3:
            val = operand
        case 4:
            val = registers['A']
        case 5:
            val = registers['B']
        case 6:
            val = registers['C']
        case _:
            raise ValueError(f'Invalid operand: {operand}')
    registers['B'] = val % 8
    return insptr+2

In [7]:
def jnz(operand, registers, insptr, outs):
    if 0 == registers['A']:
        return insptr+2
    return operand

In [8]:
def bxc(operand, registers, insptr, outs):
    registers['B'] = registers['B'] ^ registers['C']
    return insptr+2

In [9]:
def out(operand, registers, insptr, outs):
    match operand:
        case 0 | 1 | 2 | 3:
            val = operand
        case 4:
            val = registers['A']
        case 5:
            val = registers['B']
        case 6:
            val = registers['C']
        case _:
            raise ValueError(f'Invalid operand: {operand}')
    outs.append(val % 8)
    return insptr+2

In [10]:
opcodes = {0: adv, 1: bxl, 2: bst, 3: jnz, 4: bxc, 5: out, 6: bdv, 7: cdv}

In [11]:
def run(registers, program):
    outs = []
    insptr = 0
    while 0 <= insptr < len(program):
        opcode, operand = program[insptr:insptr+2]
        insptr = opcodes[opcode](operand, registers, insptr, outs)
    return ','.join(str(out) for out in outs)

In [12]:
run({'A': 0, 'B': 0, 'C': 9}, (2, 6))

''

In [13]:
run({'A': 10, 'B': 0, 'C': 0}, (5, 0, 5, 1, 5, 4))

'0,1,2'

In [14]:
run({'A': 2024, 'B': 0, 'C': 0}, (0,1,5,4,3,0))

'4,2,5,6,7,7,7,7,3,1,0'

In [15]:
run({'A': 0, 'B': 29, 'C': 0}, (1,7))

''

In [16]:
run({'A': 0, 'B': 2024, 'C': 43690}, (4,0))

''

In [17]:
def part1(data):
    registers = {'A': data['A'], 'B': data['B'], 'C': data['C']}
    program = data['Program']
    return run(registers, program)

In [18]:
part1(testdata)

'4,6,3,5,6,3,5,2,1,0'

In [19]:
part1(data)

'2,7,6,5,6,0,2,3,1'

## Part 2 ##

What value of register A will print out the original program?

In [20]:
program = data['Program']
len(program)

16

In [21]:
min_A, max_A = 8**(len(program) - 1), 8**(len(program))
min_A, max_A

(35184372088832, 281474976710656)

Brute force won't work for the puzzle data, although it will for the test data for part 2. Sneaky!

Let's simplify my puzzle program to a just be the specific code for it:

In [22]:
def myprog(A):
    outs = []
    while A > 0:
        B = A % 8
        B = B ^ 5
        C = A // (2**B)
        B = B ^ 6
        B = B ^ C
        outs.append(B % 8)
        A = A // 8
    return outs

In [23]:
myprog(33940147)

[2, 7, 6, 5, 6, 0, 2, 3, 1]

In [24]:
for A in range(min_A, max_A, 8**14):
    print(A, myprog(A))

35184372088832 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 2]
39582418599936 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 2, 2]
43980465111040 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 1, 1, 2]
48378511622144 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 1, 0, 2]
52776558133248 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 1, 2]
57174604644352 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 0, 3, 2]
61572651155456 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 0, 4, 2]
65970697666560 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 0, 7, 2]
70368744177664 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 3, 1]
74766790688768 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 7, 3, 1]
79164837199872 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 7, 1, 1]
83562883710976 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 7, 0, 1]
87960930222080 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 5, 1]
92358976733184 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 6, 3, 1]
96757023244288 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 6, 7, 1]
101155069755392 [3, 3, 3, 3, 3, 3, 3, 3,

Note that the pattern mentioned in https://www.reddit.com/r/adventofcode/comments/1hg38ah/comment/m2vn8nx/ doesn't hold. The last 4 digits change. When using 8**13, the last digit still changes. Despite that, the solution actually works. Stolen from https://github.com/ading2210/advent-of-code-solutions/blob/main/2024/day17/day17.py

In [30]:
def part2(program):
  output = []
  matched = program[-1:] #the last n digits of the program that it looks for
  init_a = 8 ** 15 #this is the minimum value required to have a 16 digit output
  power = 14 #increment by 8 ** 13 to begin with

  while output != program:
    init_a += 8 ** power
    output = myprog(init_a)
    #when the digits match, decrement the power by 1
    #by decreasing the power, the matched digits will no longer change
    if output[-len(matched):] == matched:
      print(init_a, power, matched)
      power = max(0, power - 1)
      matched = program[-(len(matched)+1):]

  return init_a

In [31]:
program = list(data['Program'])

In [32]:
program

[2, 4, 1, 5, 7, 5, 1, 6, 4, 2, 5, 5, 0, 3, 3, 0]

In [33]:
program[-1:]

[0]

In [34]:
part2(program)

105553116266496 14 [0]
106102872080384 13 [3, 0]
107202383708160 12 [3, 3, 0]
107408542138368 11 [0, 3, 3, 0]
107409615880192 10 [5, 0, 3, 3, 0]
107412837105664 9 [5, 5, 0, 3, 3, 0]
107412853882880 8 [2, 5, 5, 0, 3, 3, 0]
107412887437312 7 [4, 2, 5, 5, 0, 3, 3, 0]
107412900020224 6 [6, 4, 2, 5, 5, 0, 3, 3, 0]
107412901855232 5 [1, 6, 4, 2, 5, 5, 0, 3, 3, 0]
107416868487168 4 [5, 1, 6, 4, 2, 5, 5, 0, 3, 3, 0]
107416870453248 3 [7, 5, 1, 6, 4, 2, 5, 5, 0, 3, 3, 0]
107416870455296 2 [5, 7, 5, 1, 6, 4, 2, 5, 5, 0, 3, 3, 0]
107416870455424 1 [1, 5, 7, 5, 1, 6, 4, 2, 5, 5, 0, 3, 3, 0]
107416870455448 0 [4, 1, 5, 7, 5, 1, 6, 4, 2, 5, 5, 0, 3, 3, 0]
107416870455451 0 [2, 4, 1, 5, 7, 5, 1, 6, 4, 2, 5, 5, 0, 3, 3, 0]


107416870455451